In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import pandas as pd
import random
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from IPython.display import clear_output
from scipy.interpolate import interp1d
from datetime import datetime
import os

import numpy as np
from tensorflow import keras
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
tfd = tfp.distributions

import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten

from photoz_utils import *
from photoz_plots import *

Num GPUs Available:  1


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
z_max = 2.5

train_array = pd.read_csv('/data/HSC/HSC_v6/3_model_comparison/training_'+str(z_max)+'z_.csv') # zmax = 2.5
train_array = np.asarray(train_array)

test_array = pd.read_csv('/data/HSC/HSC_v6/3_model_comparison/testing_'+str(z_max)+'z_.csv')
test_array = np.asarray(test_array)

In [4]:
OID_test = test_array[:,0]
X_test = test_array[:,[12,13,14,15,16]]
y_test = test_array[:,2]

In [5]:
sample_array = "[1, 1, 1, 1]"

In [6]:
nnetwork = 6

In [7]:
X_total = list(train_array[:,[12,13,14,15,16]])
scaler = preprocessing.StandardScaler()
scaler.fit(X_total)

StandardScaler()

In [8]:
nensemble = 6

In [9]:
X = []
y = []
sampling_method = 'oversampled'

for i in range(1, nnetwork + 1):
    if sampling_method == 'control':
        training_array_bin = pd.read_csv('/data/HSC/HSC_v6/3_model_comparison/'+str(sampling_method)+'_training_set_'+str(z_max)+'z#1.csv')
    else:
        training_array_bin = pd.read_csv('/data/HSC/HSC_v6/3_model_comparison/'+str(sampling_method)+'_training_set_'+str(z_max)+'z_'+str(sample_array)+'#'+str(i)+'.csv')
    training_array_bin = np.asarray(training_array_bin)
    X_bin = list(training_array_bin[:,[12,13,14,15,16]])
    X_bin = scaler.transform(X_bin)
    y_bin = list(training_array_bin[:,2])
    X.append(X_bin)
    y.append(y_bin)
X = np.asarray(X)
y = np.asarray(y)

In [10]:
models = []

for i in range(0,nensemble):
    input_ = tf.keras.layers.Input(shape=X[0].shape[1:])
    hidden1 = tf.keras.layers.Dense(200, activation="tanh")(input_)
    hidden2 = tf.keras.layers.Dense(200, activation="tanh")(hidden1)
    hidden3 = tf.keras.layers.Dense(200, activation="tanh")(hidden2)
    hidden4 = tf.keras.layers.Dense(200, activation="relu")(hidden3)
    concat = tf.keras.layers.Concatenate()([input_, hidden4])
    output = tf.keras.layers.Dense(1)(concat)
    model = tf.keras.Model(inputs=[input_], outputs=[output])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse",metrics=[tf.keras.metrics.MeanAbsoluteError()])
    models.append(model)

In [ ]:
for i in range(0, nensemble):
    models[i].fit(X[i],y[i],epochs=500,shuffle = True,verbose=1)

Epoch 1/500
19686/19686 [==============================] - 33s 2ms/step - loss: 1.3319 - mean_absolute_error: 0.2403
Epoch 2/500
19686/19686 [==============================] - 32s 2ms/step - loss: 0.3190 - mean_absolute_error: 0.1936
Epoch 3/500
19686/19686 [==============================] - 32s 2ms/step - loss: 0.2648 - mean_absolute_error: 0.1772
Epoch 4/500
19686/19686 [==============================] - 32s 2ms/step - loss: 0.2742 - mean_absolute_error: 0.1696
Epoch 5/500
19686/19686 [==============================] - 32s 2ms/step - loss: 0.2470 - mean_absolute_error: 0.1636
Epoch 6/500
19686/19686 [==============================] - 32s 2ms/step - loss: 0.2271 - mean_absolute_error: 0.1586
Epoch 7/500
19686/19686 [==============================] - 32s 2ms/step - loss: 0.2501 - mean_absolute_error: 0.1552
Epoch 8/500
19686/19686 [==============================] - 32s 2ms/step - loss: 0.2345 - mean_absolute_error: 0.1509
Epoch 9/500
19686/19686 [==============================] - 32s 2

In [ ]:
y_predict = []
X_test = scaler.transform(X_test)

for i in range(0, nensemble):
    y_predict_single = models[i].predict(X_test)
    y_predict.append(np.transpose(y_predict_single)[0])

y_predict = np.transpose(y_predict)

In [ ]:
mean_array = []
std_array = []
for i in range(0,len(X_test)):
    mean = np.mean(y_predict[i])
    std = np.std(y_predict[i])
    mean_array.append(mean)
    std_array.append(std)

In [ ]:
plot_predictions(mean_array, y_test)

In [ ]:
sample_array_string = ""
for i in sample_array:
    sample_array_string += str(i)
    sample_array_string += "_"

ensemble_prediction = np.transpose(np.vstack((OID_test, y_test, mean_array, std_array)))

df = pd.DataFrame(ensemble_prediction, columns=['object_id', 'specz_redshift', 'photoz_redshift',
       'photoz_uncertainty'])
model_name = 'ensemble_nn_v1'
metrics = get_point_metrics(pd.Series(mean_array), pd.Series(y_test), 2.5, binned=False)

if os.path.exists(f'/predictions/HSC_v6_sampling_method_models/{model_name}') == False:
    os.makedirs(f'/predictions/HSC_v6_sampling_method_models/{model_name}')
if sampling_method == 'control':
    df.to_csv(f'/predictions/HSC_v6_sampling_method_models/{model_name}/{sampling_method}_{z_max}z_predictions.csv', index=False)
    metrics.to_csv(f'/predictions/HSC_v6_sampling_method_models/{model_name}/{sampling_method}_{z_max}z_metrics.csv', index=False)
else:
    df.to_csv(f'/predictions/HSC_v6_sampling_method_models/{model_name}/{sampling_method}_{z_max}z_predictions_{sample_array}.csv', index=False)
    metrics.to_csv(f'/predictions/HSC_v6_sampling_method_models/{model_name}/{sampling_method}_{z_max}z_metrics_{sample_array}.csv', index=False)